<strong>QuickSight Folder Migration</strong>

Author: Ian Liao (Data Visualization Engineer in ProServe GSP)
Date: Sep 13 2021

QuickSight Folder can be used to build deployment environments. User can create DEV, UAT and PROD folders, and use them as corresponding environments.
This migration script helps to dashboards with dependencies from one folder to another. 
Folders can be in the same QuickSight account, or different QuickSight accounts.

<strong>Requirements</strong>

The Dashboard name has to be unique globally</br>
Folder name has be to unique globally. </br>
Always use CAPITAL LETTERS ONLY NO NUMBERS NO SPECIAL CHARACTERS for folder name. </br>
A DEV folder has to exist, and it is used as the base folder (or referred as main). </br>
Never migrate backwards, start a new development folder for hot-fix if necessary. Migrate to development folder creates an analysis instead of a dashboard </br> 
DO NOT use dash in dataset name </br>

<strong>How it works?</strong>

The script will try to find the specified dashboards in the source folder, find their datasets, and migrate to destination folder. </br>
Object migrated to any folder other than DEV will use "its ID in DEV - folder name" as the new ID. </br>

<strong>What objects are migrated</strong>

In same account migration, Datasets and Dashboards are migrated; </br>
In cross account migration, Datasets, Themes and Dashboards are migrated. </br>
In either case, data source is not migrated. It assumes only one data source, and user should provide the datasource id in the target account

<strong>Limitations</strong>

Dataset refresh schedule is not set in target environment </br>
Migrate from HF to UAT will create a new dashboard. Developer needs to delete or remove the older version dashboard manually.




In [1]:
!pip install --upgrade pip
!pip install --upgrade boto3
get_ipython().system('pip install --upgrade ipynb')

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 1.7 MB 28.9 MB/s            
  Attempting uninstall: pip
    Found existing installation: pip 21.3
    Uninstalling pip-21.3:
      Successfully uninstalled pip-21.3
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 131 kB 41.2 MB/s            
     |████████████████████████████████| 8.0 MB 57.4 MB/s            
  Attempting uninstall: botocore
    Found existing installation: botocore 1.21.61
    Uninstalling botocore-1.21.61:
      Successfully uninstalled botocore-1.21.61
  Attempting uninstall: boto3
    Found existing installation: boto3 1.18.61
    Uninstalling boto3-1.18.61:
      Successfully uninstalled boto3-1.18.61
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conf

In [2]:
'''
#DEV -> UAT
# Configure the migration scripts

# dashboard migration list
dashboard_migrate_list = ['<name of dashboards to migrate>']
theme_migrate_list = [] # provide theme name for cross account migration

# source info
sourceaccountid='<source_account_name>'
source_role_name='<execution role for the source account>'
source_aws_region='us-east-1'
source_folder_name='DEV'
source_folder_ID='<ID of the DEV folder>'
source_is_dev = True # analysis exists in dev folder
source_is_main = True # in main environment, object IDs don't have folder name suffix
source_admin_name = '<name of QuickSight admin user>'


# target info
targetaccountid='<target_account_name>'
target_role_name='<execution role for the account>'
target_aws_region='us-east-1'
target_folder_name='UAT'
target_folder_ID='ID of the UAT folder'
target_is_dev = False # set this flag to true if analysis migration is necessary (in use case like roll back); by default analysis will not be migrated. 
target_is_main = False
target_admin_name = '<name of QuickSight admin user>'

target_datasource_id = '' # provide target data source id for cross account migration

# migration settings
same_account_migration = (True if sourceaccountid == targetaccountid else False)

'''

In [3]:
'''
# UAT -> HF


# Configure the migration scripts

# dashboard migration list
dashboard_migrate_list = ['<name of dashboards to migrate>'] # name of the dashboard
theme_migrate_list = [] # provide theme name for cross account migration

# source info
sourceaccountid='<source_account_name>'
source_role_name='<execution role for the source account>'
source_aws_region='us-east-1'
source_folder_name='UAT'
source_folder_ID='<ID of the UAT folder>'
source_is_dev = False # analysis exists in dev folder
source_is_main = False # in main environment, object IDs don't have folder name suffix
source_admin_name = '<name of QuickSight admin user>'


# target info
targetaccountid='233081732471'
target_role_name='<execution role for the account>'
target_aws_region='us-east-1'
target_folder_name='HF'
target_folder_ID='<ID of the HF folder>'
target_is_dev = True # set this flag to true if analysis migration is necessary (in use case like roll back); by default analysis will not be migrated. 
target_is_main = False
target_admin_name = '<name of QuickSight admin user>'

target_datasource_id = '' # provide target data source id for cross account migration

# migration settings
same_account_migration = (True if sourceaccountid == targetaccountid else False)
'''

"\n# UAT -> HF\n\n\n# Configure the migration scripts\n\n# dashboard migration list\ndashboard_migrate_list = ['Dealer Analytic Dashboard-UAT'] # name of the dashboard\ntheme_migrate_list = [] # provide theme name for cross account migration\n\n# source info\nsourceaccountid='233081732471'\nsource_role_name='SageMakerExecutionRoleQuickSightMigration'\nsource_aws_region='us-east-1'\nsource_folder_name='UAT'\nsource_folder_ID='9e1c0f70-21ec-48da-a50f-1a0c7fd12075'\nsource_is_dev = False # analysis exists in dev folder\nsource_is_main = False # in main environment, object IDs don't have folder name suffix\nsource_admin_name = 'VGCS-QS/david.mcneil'\n\n\n# target info\ntargetaccountid='233081732471'\ntarget_role_name='SageMakerExecutionRoleQuickSightMigration'\ntarget_aws_region='us-east-1'\ntarget_folder_name='HF'\ntarget_folder_ID='2e555c31-4345-4c58-b4d3-7460f8c21301'\ntarget_is_dev = True # set this flag to true if analysis migration is necessary (in use case like roll back); by defaul

In [4]:
'''
#HF -> UAT
# Configure the migration scripts

# dashboard migration list
dashboard_migrate_list = ['<name of dashboards to migrate>']
theme_migrate_list = [] # provide theme name for cross account migration

# source info
sourceaccountid='<source_account_name>'
source_role_name='<execution role for the source account>'
source_aws_region='us-east-1'
source_folder_name='HF'
source_folder_ID='<ID of the HF folder>'
source_is_dev = True # analysis exists in dev folder
source_is_main = False # in main environment, object IDs don't have folder name suffix
source_admin_name = '<name of QuickSight admin user>'


# target info
targetaccountid='233081732471'
target_role_name='<execution role for the account>'
target_aws_region='us-east-1'
target_folder_name='UAT'
target_folder_ID='<ID of the UAT folder>'
target_is_dev = False # set this flag to true if analysis migration is necessary (in use case like roll back); by default analysis will not be migrated. 
target_is_main = False
target_admin_name = '<name of QuickSight admin user>'

target_datasource_id = '' # provide target data source id for cross account migration

# migration settings
same_account_migration = (True if sourceaccountid == targetaccountid else False)
'''

"\n#HF -> UAT\n# Configure the migration scripts\n\n# dashboard migration list\ndashboard_migrate_list = ['NEWHF']\ntheme_migrate_list = [] # provide theme name for cross account migration\n\n# source info\nsourceaccountid='233081732471'\nsource_role_name='SageMakerExecutionRoleQuickSightMigration'\nsource_aws_region='us-east-1'\nsource_folder_name='HF'\nsource_folder_ID='2e555c31-4345-4c58-b4d3-7460f8c21301'\nsource_is_dev = True # analysis exists in dev folder\nsource_is_main = False # in main environment, object IDs don't have folder name suffix\nsource_admin_name = 'VGCS-QS/david.mcneil'\n\n\n# target info\ntargetaccountid='233081732471'\ntarget_role_name='SageMakerExecutionRoleQuickSightMigration'\ntarget_aws_region='us-east-1'\ntarget_folder_name='UAT'\ntarget_folder_ID='9e1c0f70-21ec-48da-a50f-1a0c7fd12075'\ntarget_is_dev = False # set this flag to true if analysis migration is necessary (in use case like roll back); by default analysis will not be migrated. \ntarget_is_main = F

In [5]:
'''#UAT -> DEV
# a rare use case to reset main branch
# Don't run it unless absolute necessary
# Configure the migration scripts


# dashboard migration list
dashboard_migrate_list = ['<name of dashboards to migrate>'] # name of the dashboard
theme_migrate_list = [] # provide theme name for cross account migration

# source info
sourceaccountid='<source_account_name>'
source_role_name='<execution role for the source account>'
source_aws_region='us-east-1'
source_folder_name='UAT'
source_folder_ID='<ID of the UAT folder>'
source_is_dev = False # analysis exists in dev folder
source_is_main = False # in main environment, object IDs don't have folder name suffix
source_admin_name = '<name of QuickSight admin user>'


# target info
targetaccountid='233081732471'
target_role_name='<execution role for the account>'
target_aws_region='us-east-1'
target_folder_name='DEV'
target_folder_ID='<ID of the DEV folder>'
target_is_dev = True # set this flag to true if analysis migration is necessary (in use case like roll back); by default analysis will not be migrated. 
target_is_main = True # a rare use case to reset main branch
target_admin_name = '<name of QuickSight admin user>'

target_datasource_id = '' # provide target data source id for cross account migration

# migration settings
same_account_migration = (True if sourceaccountid == targetaccountid else False)'''

"#UAT -> DEV\n# a rare use case to reset main branch\n# Don't run it unless absolute necessary\n# Configure the migration scripts\n\n\n# dashboard migration list\ndashboard_migrate_list = ['NEWHF'] # name of the dashboard\ntheme_migrate_list = [] # provide theme name for cross account migration\n\n# source info\nsourceaccountid='233081732471'\nsource_role_name='SageMakerExecutionRoleQuickSightMigration'\nsource_aws_region='us-east-1'\nsource_folder_name='UAT'\nsource_folder_ID='9e1c0f70-21ec-48da-a50f-1a0c7fd12075'\nsource_is_dev = False # analysis exists in dev folder\nsource_is_main = False # in main environment, object IDs don't have folder name suffix\nsource_admin_name = 'VGCS-QS/david.mcneil'\n\n\n# target info\ntargetaccountid='233081732471'\ntarget_role_name='SageMakerExecutionRoleQuickSightMigration'\ntarget_aws_region='us-east-1'\ntarget_folder_name='DEV'\ntarget_folder_ID='d6e161df-07bc-4d76-a3e0-99b0e0fafb18'\ntarget_is_dev = True # set this flag to true if analysis migrati

In [6]:
'''# create config dict
config = {}
config['dashboard_migrate_list'] = dashboard_migrate_list
# source info
config['sourceaccountid'] = sourceaccountid
config['source_role_name']= source_role_name
config['source_aws_region']= source_aws_region
config['source_folder_name']= source_folder_name
config['source_folder_ID']= source_folder_ID
config['source_is_dev'] = source_is_dev
config['source_is_main'] = source_is_main
config['source_admin_name'] = source_admin_name


# target info
config['targetaccountid']=targetaccountid
config['target_role_name']=target_role_name
config['target_aws_region']=target_aws_region
config['target_folder_name']=target_folder_name
config['target_folder_ID']=target_folder_ID
config['target_is_dev'] = target_is_dev
config['target_admin_name'] = target_admin_name

# migration settings
config['same_account_migration'] = same_account_migration'''

"# create config dict\nconfig = {}\nconfig['dashboard_migrate_list'] = dashboard_migrate_list\n# source info\nconfig['sourceaccountid'] = sourceaccountid\nconfig['source_role_name']= source_role_name\nconfig['source_aws_region']= source_aws_region\nconfig['source_folder_name']= source_folder_name\nconfig['source_folder_ID']= source_folder_ID\nconfig['source_is_dev'] = source_is_dev\nconfig['source_is_main'] = source_is_main\nconfig['source_admin_name'] = source_admin_name\n\n\n# target info\nconfig['targetaccountid']=targetaccountid\nconfig['target_role_name']=target_role_name\nconfig['target_aws_region']=target_aws_region\nconfig['target_folder_name']=target_folder_name\nconfig['target_folder_ID']=target_folder_ID\nconfig['target_is_dev'] = target_is_dev\nconfig['target_admin_name'] = target_admin_name\n\n# migration settings\nconfig['same_account_migration'] = same_account_migration"

In [7]:
import boto3
import json
import time
from IPython.display import JSON
import sys
import ipynb.fs 
import logging
from typing import Any, Dict, List, Optional
from datetime import datetime
import re

# current date and time
now = str(datetime.now().strftime("%m-%d-%Y_%H_%M"))

Import functions from functions notebook

In [8]:
from ipynb.fs.defs.Functions import data_sources
from ipynb.fs.defs.Functions import describe_source 
from ipynb.fs.defs.Functions import delete_source
from ipynb.fs.defs.Functions import create_data_source
from ipynb.fs.defs.Functions import get_datasource_name
from ipynb.fs.defs.Functions import get_datasource_ids
from ipynb.fs.defs.Functions import update_data_source_permissions

from ipynb.fs.defs.Functions import get_dataset_name
from ipynb.fs.defs.Functions import data_sets
from ipynb.fs.defs.Functions import describe_dataset
from ipynb.fs.defs.Functions import get_dataset_ids
from ipynb.fs.defs.Functions import delete_dataset 
from ipynb.fs.defs.Functions import create_dataset
from ipynb.fs.defs.Functions import update_dataset
from ipynb.fs.defs.Functions import update_data_set_permissions

from ipynb.fs.defs.Functions import get_target

from ipynb.fs.defs.Functions import templates
from ipynb.fs.defs.Functions import delete_template
from ipynb.fs.defs.Functions import update_template_permission 
from ipynb.fs.defs.Functions import copy_template
from ipynb.fs.defs.Functions import describe_template
from ipynb.fs.defs.Functions import create_template 

from ipynb.fs.defs.Functions import dashboards
from ipynb.fs.defs.Functions import describe_dashboard
from ipynb.fs.defs.Functions import create_dashboard 
from ipynb.fs.defs.Functions import delete_dashboard
from ipynb.fs.defs.Functions import update_dashboard 
from ipynb.fs.defs.Functions import get_dashboard_ids
from ipynb.fs.defs.Functions import get_dashboard_name

from ipynb.fs.defs.Functions import themes
from ipynb.fs.defs.Functions import describe_theme
from ipynb.fs.defs.Functions import delete_theme
from ipynb.fs.defs.Functions import create_theme
from ipynb.fs.defs.Functions import update_theme
from ipynb.fs.defs.Functions import describe_theme_permissions
from ipynb.fs.defs.Functions import update_theme_permissions

from ipynb.fs.defs.Functions import analysis
from ipynb.fs.defs.Functions import describe_analysis
from ipynb.fs.defs.Functions import create_analysis
from ipynb.fs.defs.Functions import delete_analysis
from ipynb.fs.defs.Functions import update_analysis
from ipynb.fs.defs.Functions import get_analysis_ids
from ipynb.fs.defs.Functions import describe_analysis_permissions


from ipynb.fs.defs.Functions import folder_members

#supportive functions
from ipynb.fs.defs.Functions import data_sets_ls_of_dashboard
from ipynb.fs.defs.Functions import data_sources_ls_of_dashboard
from ipynb.fs.defs.Functions import get_data_source_migration_list
from ipynb.fs.defs.Functions import data_sources_ls_of_analysis
from ipynb.fs.defs.Functions import data_sets_ls_of_analysis
from ipynb.fs.defs.Functions import get_user_arn
from ipynb.fs.defs.Functions import _assume_role

from ipynb.fs.defs.Functions import create_group_membership

<strong>Static Profile</strong>

You can also configure AWS profile from terminal and call the profile in below cell

In [9]:
'''
sourceprofile=''
targetprofile=''
sourcesession = boto3.Session(profile_name=sourceprofile, region_name=source_aws_region)
targetsession = boto3.Session(profile_name=targetprofile, region_name=target_aws_region)
'''

"\nsourceprofile=''\ntargetprofile=''\nsourcesession = boto3.Session(profile_name=sourceprofile, region_name=source_aws_region)\ntargetsession = boto3.Session(profile_name=targetprofile, region_name=target_aws_region)\n"

In [10]:
sourceprofile=''
targetprofile=''
sourcesession = boto3.Session(region_name=source_aws_region)
targetsession = boto3.Session(region_name=target_aws_region)

<strong>Assume Role</strong>

You can also assume an IAM role and create session based on the role permissions

In [11]:
'''#source account
sourcesession = _assume_role(sourceaccountid, source_role_name, source_aws_region)

#target account
targetsession = _assume_role(targetaccountid, target_role_name, target_aws_region)
#targetsession = boto3.Session(
#        aws_access_key_id="",
#        aws_secret_access_key="",
#        aws_session_token="",
#        region_name=aws_region
#    )'''

'#source account\nsourcesession = _assume_role(sourceaccountid, source_role_name, source_aws_region)\n\n#target account\ntargetsession = _assume_role(targetaccountid, target_role_name, target_aws_region)\n#targetsession = boto3.Session(\n#        aws_access_key_id="",\n#        aws_secret_access_key="",\n#        aws_session_token="",\n#        region_name=aws_region\n#    )'

<strong>Set root and admin users</strong>

root user is for the template. 
By default, we assign full permissions of objects to admin.

In [12]:
sourceroot=get_user_arn (sourcesession, 'root')
sourceadmin=get_user_arn (sourcesession, source_admin_name)
#sourceversion='1'

targetroot=get_user_arn (targetsession, 'root')
targetadmin=get_user_arn (targetsession, target_admin_name)
#targetvpc='arn:aws:quicksight:us-east-1:889399602426:vpcConnection/sg-40b7521a'

<strong>Please define your input parameters in below cell</strong>

In [13]:
# data source credentials. May use in secret manager in future. 

rds='mssql'
redshift={
    "ClusterId": 'wangzyncluster1',
    "Host": 'wangzyncluster1.coprq8ycemvc.us-east-1.redshift.amazonaws.com',
    "Database": 'dev'}

s3Bucket='spaceneedle-samplefiles.prod.us-east-1'
s3Key='sales/manifest.json'
vpc='sg-40b7521a'
tag=[
        {
            'Key': 'covid-19-dashboard-migration',
            'Value': 'true'
        }
    ]
owner=targetadmin
rdscredential={
        'CredentialPair': {
            'Username': "",
            'Password': ""}}
redshiftcredential={
        'CredentialPair': {
            'Username': "ro_user",
            'Password': "Ro_user1234"}}
region='us-east-1'
namespace='default'
version='1' 

target=get_target(targetsession, rds,redshift,s3Bucket,s3Key,vpc,tag,owner,rdscredential,redshiftcredential)

JSON(target)


<IPython.core.display.JSON object>

<strong>Migration List<strong/>

In [14]:
if same_account_migration == True:
    migrate_p = 'dashboard'
    source_migrate_list = []
    theme_migrate_list= []
else:
    migrate_p = 'dashboard'
    source_migrate_list = []
    
 
""""
"all" will migrate data source, dataset, theme, analysis and dashboard;
"source" means data sources only; 
"dataset" means datasets only; 
"theme" means theme only;
"analysis" means analysis only;
"dashboard" means dashboard only
""" 

dataset_migrate_list = []
analysis_migrate_list= []
# dashboard migration list is now defined on top
#dashboard_migrate_list = ['patient'] 

In [15]:
if migrate_p in ['dashboard']:
    source_migrate_list=[]
    dataset_migrate_list=[]
    for dashboard in dashboard_migrate_list:
        print(dashboard)
        #datasources=data_sources_ls_of_dashboard(dashboard, sourcesession)
        #print(datasources)
        #for datasource in datasources:
        #    source_migrate_list.append(datasource)
        datasets=data_sets_ls_of_dashboard(dashboard, sourcesession)
        for dataset in datasets:
            dataset_migrate_list.append(dataset)
            
if migrate_p in ['analysis']:
    source_migrate_list=[]
    dataset_migrate_list=[]
    for analysis_name in analysis_migrate_list:
        print(analysis_name)
        datasources=data_sources_ls_of_analysis(analysis_name, sourcesession)
        print(datasources)
        for datasource in datasources:
            source_migrate_list.append(datasource)
        datasets=data_sets_ls_of_analysis(analysis_name, sourcesession)
        print(datasets)
        for dataset in datasets:
            dataset_migrate_list.append(dataset)
    
if migrate_p in ['all']:
    for dashboard in dashboard_migrate_list:
        datasources=data_sources_ls_of_dashboard(dashboard, sourcesession)
        for datasource in datasources:
            source_migrate_list.append(datasource)
        datasets=data_sets_ls_of_dashboard(dashboard, sourcesession)
        for dataset in datasets:
            dataset_migrate_list.append(dataset)
            
    for analysis_name in analysis_migrate_list:
        datasources=data_sources_ls_of_analysis(analysis_name, sourcesession)
        for datasource in datasources:
            source_migrate_list.append(datasource)
        datasets=data_sets_ls_of_analysis(analysis_name, sourcesession)
        for dataset in datasets:
            dataset_migrate_list.append(dataset)

Demo Analytics Dashboard
['b22fbc21-bf52-40fb-803c-8469424a7dd2']


In [16]:
dashboard_migrate_list

['Demo Analytics Dashboard']

In [17]:
dataset_migrate_list

['qs_subscription',
 'qs_faults',
 'qs_ecu',
 'qs_vehicle_exclusion',
 'qs_dataset_date_range',
 'qs_fuel_comparison',
 'qs_veh_dimension',
 'qs_location',
 'qs_software',
 'qs_fuel',
 'qs_not_reporting',
 'qs_location_comparison',
 'qs_geofence']

In [18]:
source_migrate_list

[]

In [19]:
theme_migrate_list

[]

In [20]:
analysis_migrate_list

[]

In [21]:
def get_target_id(source_id):
    if source_is_main:
        target_id = source_id + '-' + target_folder_name
    elif target_is_main:
        target_id = source_id.replace(('-' + source_folder_name), '')
    else:
        target_id = source_id.replace(('-' + source_folder_name), '') + ('-' + target_folder_name)
        
    return target_id

In [22]:
def get_target_placeholder(placeholder):
    if target_is_main:
        target_placeholder = re.sub(r'-[A-Z]+', '', placeholder)
    else:
        target_placeholder = re.sub(r'-[A-Z]+', '', placeholder) + '-' + target_folder_name
    return target_placeholder

<strong>Results Output Location<strong/>

In [23]:
#
successlocation = "Migration_Results/Successful/"
faillocation = "Migration_Results/Fail/"

import os
try:
    os.makedirs(successlocation)
except OSError:
    print ("Creation of the directory %s failed" % successlocation)
else:
    print ("Successfully created the directory %s" % successlocation)

try:
    os.makedirs(faillocation)
except OSError:
    print ("Creation of the directory %s failed" % faillocation)
else:
    print ("Successfully created the directory %s" % faillocation)

Creation of the directory Migration_Results/Successful/ failed
Creation of the directory Migration_Results/Fail/ failed


<strong> Data Set Migration </strong> Get datasets list:

#source account
sourceaccountid=""
role_name=""
aws_region=''
sourcesession = _assume_role(sourceaccountid, role_name, aws_region)

#target account
targetaccountid=""
role_name=""
aws_region='us-east-1'
targetsession = _assume_role(targetaccountid, role_name, aws_region)

In [24]:
datasets=data_sets(sourcesession)

migration_list=[]
for newset in dataset_migrate_list:
        ids = get_dataset_ids(newset, sourcesession)  #Get id of datasets migration list
        for dataset in datasets:
            if ids[0] == dataset["DataSetId"]:
                migration_list.append(dataset)

JSON(migration_list)

<IPython.core.display.JSON object>

In [25]:
migration_list

[{'Arn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/0e91b4ef-4a53-495d-8c88-66560abed72e',
  'DataSetId': '0e91b4ef-4a53-495d-8c88-66560abed72e',
  'Name': 'qs_subscription',
  'CreatedTime': datetime.datetime(2021, 6, 15, 15, 26, 8, 839000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2021, 10, 22, 6, 1, 15, 744000, tzinfo=tzlocal()),
  'ImportMode': 'SPICE',
  'RowLevelPermissionDataSet': {'Namespace': 'default',
   'Arn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/02f11126-608f-4b6c-8ec5-e29f8e29dc8d',
   'PermissionPolicy': 'GRANT_ACCESS',
   'FormatVersion': 'VERSION_1',
   'Status': 'ENABLED'},
  'RowLevelPermissionTagConfigurationApplied': False,
  'ColumnLevelPermissionRulesApplied': False},
 {'Arn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/1d04d9b5-d25f-4fd6-bb28-60392e25a6be',
  'DataSetId': '1d04d9b5-d25f-4fd6-bb28-60392e25a6be',
  'Name': 'qs_faults',
  'CreatedTime': datetime.datetime(2021, 6, 15, 18, 42, 30, 337000, tzinfo=tzlocal())

<strong>Get already migrated datasets list</strong>

In [26]:
#get datasets which already migrated
targetds=data_sets(targetsession)
#already_migrated record the datasets ids of target account
already_migrated=[]
for ds in targetds:
    already_migrated.append(ds['DataSetId'])
#already_migrated

Migrate Datasets

In [27]:
newsetslist=[]
faillist=[]
sts_client = targetsession.client("sts")
account_id = sts_client.get_caller_identity()["Account"]


In [28]:
def migrate_dataset(mds):  
    print(get_target_id(mds['DataSetId']))
    try:
        res=describe_dataset(sourcesession, mds['DataSetId'])
    except Exception:
        faillist.append({"Dataset": mds, "Error": str(Exception)})
        return
    
    if 'RowLevelPermissionDataSet' in res['DataSet']:
        rls_flag = True
        rls_arn = res['DataSet']['RowLevelPermissionDataSet']['Arn']
        print('rls data set exists: ' + rls_arn)
        rls_id = rls_arn.split('/')[1]
        
        try:
            rls_res = describe_dataset(sourcesession, rls_id)
        except Exception:
            faillist.append({"Dataset": mds, "RLS": rls_id, "Error": str(Exception)})
            return
            
        rls_name = rls_res['DataSet']['Name']
        
        rls = {}
        rls['DataSetId'] = rls_id
        rls['Name'] = rls_name
        
        print(rls)
        #migrate the rls dataset to target
        migrate_dataset(rls)
        already_migrated.append(get_target_id(rls_id))
        
        RowLevelPermissionDataSet = res['DataSet']['RowLevelPermissionDataSet']
        RowLevelPermissionDataSet['Arn'] = get_target_id(RowLevelPermissionDataSet['Arn'])
        RowLevelPermissionDataSet['Status'] = 'ENABLED'
        
    else:
        rls_flag = False
        RowLevelPermissionDataSet = None
        
    
    
    name=mds['Name']
    datasetid=mds['DataSetId']
    
    PT=res['DataSet']['PhysicalTableMap']
    for key, value in PT.items():
        for k,v in value.items():
            dsid = v['DataSourceArn'].split("/")[1] if same_account_migration else target_datasource_id
            v['DataSourceArn']='arn:aws:quicksight:us-east-1:'+account_id+':datasource/'+dsid

    LT=res['DataSet']['LogicalTableMap']
    if 'ColumnGroups' in res['DataSet']:
        ColumnGroups=res['DataSet']['ColumnGroups']
    else: ColumnGroups=None
    
    if get_target_id(mds['DataSetId']) not in already_migrated:
        try: 
            newdataset=create_dataset(targetsession, get_target_id(datasetid), get_target_id(name), PT, LT, res['DataSet']['ImportMode'], target['datasetpermission'],ColumnGroups, RowLevelPermissionDataSet)
            print("new dataset: ", newdataset)
            newsetslist.append(newdataset)
        except Exception as e:
            print('failed: '+str(e))
            faillist.append({"DataSetId": datasetid, "Name": name, "Error": str(e)})
            return
            
    if get_target_id(mds['DataSetId']) in already_migrated:
        try: 
            newdataset=update_dataset(targetsession, get_target_id(datasetid), get_target_id(name), PT, LT, res['DataSet']['ImportMode'],ColumnGroups, RowLevelPermissionDataSet)
            print("update dataset: ", newdataset)
            newsetslist.append(newdataset)
        except Exception as e:
            print('failed: '+str(e))
            faillist.append({"DataSetId": datasetid, "Name": name, "Error": str(e)})
            return

    create_group_membership(targetsession, target_folder_ID, get_target_id(datasetid), 'DATASET' )        
    print('added DATASET {} to target folder'.format(get_target_id(datasetid)))


In [29]:
migration_list

[{'Arn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/0e91b4ef-4a53-495d-8c88-66560abed72e',
  'DataSetId': '0e91b4ef-4a53-495d-8c88-66560abed72e',
  'Name': 'qs_subscription',
  'CreatedTime': datetime.datetime(2021, 6, 15, 15, 26, 8, 839000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2021, 10, 22, 6, 1, 15, 744000, tzinfo=tzlocal()),
  'ImportMode': 'SPICE',
  'RowLevelPermissionDataSet': {'Namespace': 'default',
   'Arn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/02f11126-608f-4b6c-8ec5-e29f8e29dc8d',
   'PermissionPolicy': 'GRANT_ACCESS',
   'FormatVersion': 'VERSION_1',
   'Status': 'ENABLED'},
  'RowLevelPermissionTagConfigurationApplied': False,
  'ColumnLevelPermissionRulesApplied': False},
 {'Arn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/1d04d9b5-d25f-4fd6-bb28-60392e25a6be',
  'DataSetId': '1d04d9b5-d25f-4fd6-bb28-60392e25a6be',
  'Name': 'qs_faults',
  'CreatedTime': datetime.datetime(2021, 6, 15, 18, 42, 30, 337000, tzinfo=tzlocal())

In [30]:
for mds in migration_list:
    migrate_dataset(mds)

0e91b4ef-4a53-495d-8c88-66560abed72e-UAT
rls data set exists: arn:aws:quicksight:us-east-1:233081732471:dataset/02f11126-608f-4b6c-8ec5-e29f8e29dc8d
{'DataSetId': '02f11126-608f-4b6c-8ec5-e29f8e29dc8d', 'Name': 'qs_rls_general_a'}
02f11126-608f-4b6c-8ec5-e29f8e29dc8d-UAT
update dataset:  {'ResponseMetadata': {'RequestId': 'c74d8241-d1ad-49b7-b0e7-cdfe7477debf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 22 Oct 2021 16:56:57 GMT', 'content-type': 'application/json', 'content-length': '414', 'connection': 'keep-alive', 'x-amzn-requestid': 'c74d8241-d1ad-49b7-b0e7-cdfe7477debf'}, 'RetryAttempts': 0}, 'Status': 200, 'Arn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/02f11126-608f-4b6c-8ec5-e29f8e29dc8d-UAT', 'DataSetId': '02f11126-608f-4b6c-8ec5-e29f8e29dc8d-UAT', 'IngestionArn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/02f11126-608f-4b6c-8ec5-e29f8e29dc8d-UAT/ingestion/8321bd9e-a24a-4abd-9c1b-9758fdca3d88', 'IngestionId': '8321bd9e-a24a-4abd-9c1b-9758fdca3d88', 

In [31]:
newsetslist

[{'ResponseMetadata': {'RequestId': 'c74d8241-d1ad-49b7-b0e7-cdfe7477debf',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Fri, 22 Oct 2021 16:56:57 GMT',
    'content-type': 'application/json',
    'content-length': '414',
    'connection': 'keep-alive',
    'x-amzn-requestid': 'c74d8241-d1ad-49b7-b0e7-cdfe7477debf'},
   'RetryAttempts': 0},
  'Status': 200,
  'Arn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/02f11126-608f-4b6c-8ec5-e29f8e29dc8d-UAT',
  'DataSetId': '02f11126-608f-4b6c-8ec5-e29f8e29dc8d-UAT',
  'IngestionArn': 'arn:aws:quicksight:us-east-1:233081732471:dataset/02f11126-608f-4b6c-8ec5-e29f8e29dc8d-UAT/ingestion/8321bd9e-a24a-4abd-9c1b-9758fdca3d88',
  'IngestionId': '8321bd9e-a24a-4abd-9c1b-9758fdca3d88',
  'RequestId': 'c74d8241-d1ad-49b7-b0e7-cdfe7477debf'},
 {'ResponseMetadata': {'RequestId': '0373ef7d-0dd9-43ac-851d-708d258150d6',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Fri, 22 Oct 2021 16:56:59 GMT',
    'content-type': 'application/js

In [32]:
faillist

[]

In [33]:
#print fail informations
with open(faillocation+now+'Dataset_Creation_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

successfulls=[]
for news in newsetslist:
    dataset=describe_dataset(targetsession, news['DataSetId'])
    successfulls.append(dataset['DataSet'])
    
with open(successlocation+now+'Datasets_Creation_Success.json', "w") as f:
    json.dump(successfulls, f, indent=4, sort_keys=True, default=str)


Get themes list

In [34]:
themes_list_complete =themes(sourcesession)
themes_list=[]
#JSON(datasets)
for th in themes_list_complete:
    if th["Name"] in theme_migrate_list:
        themes_list.append(th)

Migrate Themes

In [35]:
#get themes which already migrated
targetthemes=themes(targetsession)
#already_migrated record the datasets ids of target account
already_migrated=[]
for th in targetthemes:
    already_migrated.append(th['ThemeId'])
#already_migrated

In [36]:
newthemeslist=[]
faillist=[]
sts_client = targetsession.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
for i in themes_list:
    if i['ThemeId'] not in already_migrated:
        try:
            res=describe_theme(sourcesession, i['ThemeId'])
        except Exception:
            faillist.append({"Theme": i, "Error": str(Exception)})
            continue
        THEMEID=res['Theme']['ThemeId']
        Name=res['Theme']['Name']
        BaseThemeId=res['Theme']['Version']['BaseThemeId']
        Configuration=res['Theme']['Version']['Configuration']
        try: 
            newtheme=create_theme (targetsession,THEMEID, Name,BaseThemeId,Configuration)
            newthemeslist.append(newtheme)
        except Exception as e:
            #print('failed: '+str(e))
            faillist.append({"ThemeID": THEMEID, "Name": Name, "Error": str(e)})
            continue
        try:
            update_theme_permissions(targetsession, THEMEID, targetadmin)
        except Exception as e:
            #print('failed: '+str(e))
            faillist.append({"ThemeID": THEMEID, "Name": Name, "Error": str(e)})
            continue

In [37]:
#print fail informations
with open(faillocation+now+'Themes_Creation_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

successfulls=[]
for news in newthemeslist:
    theme=describe_theme(targetsession, news['ThemeId'])
    successfulls.append(theme['Theme']['ThemeId'])
    
with open(successlocation+now+'Themes_Creation_Success.json', "w") as f:
    json.dump(successfulls, f, indent=4, sort_keys=True, default=str)

Get analysis

In [38]:
sourceanalysis_list_complete=analysis(sourcesession)
sourceanalysis_list=[]
for a in sourceanalysis_list_complete:
    if a["Name"] in analysis_migrate_list:
        sourceanalysis_list.append(a)

In [39]:
sourceanalysis_list_complete

[{'Arn': 'arn:aws:quicksight:us-east-1:233081732471:analysis/02836ee7-65ec-4439-91df-2a7c00069e41',
  'AnalysisId': '02836ee7-65ec-4439-91df-2a7c00069e41',
  'Name': 'Web and Social Media Analytics analysis',
  'Status': 'CREATION_SUCCESSFUL',
  'CreatedTime': datetime.datetime(2020, 12, 22, 19, 47, 24, 135000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2020, 12, 22, 19, 47, 24, 135000, tzinfo=tzlocal())},
 {'Arn': 'arn:aws:quicksight:us-east-1:233081732471:analysis/06160945-c539-4aa9-9dd7-569bd206e53e',
  'AnalysisId': '06160945-c539-4aa9-9dd7-569bd206e53e',
  'Name': 'Sales Pipeline analysis',
  'Status': 'CREATION_SUCCESSFUL',
  'CreatedTime': datetime.datetime(2021, 4, 25, 22, 16, 26, 473000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2021, 4, 25, 22, 16, 26, 473000, tzinfo=tzlocal())},
 {'Arn': 'arn:aws:quicksight:us-east-1:233081732471:analysis/079bbebd-53a2-4b55-9a0b-dd195437863c',
  'AnalysisId': '079bbebd-53a2-4b55-9a0b-dd195437863c',
  'Name': 'p

Migrate Analysis

In [40]:
sourceanalysis_all=[]
for i in sourceanalysis_list:
    if i['Status']!= 'DELETED':
        sourceanalysis_all.append(i)

success=[]
faillist=[]
sts_client = targetsession.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
for i in sourceanalysis_all:
    sourceanalysis=describe_analysis(sourcesession, i['AnalysisId'])
    sourceanalysisid=sourceanalysis['Analysis']['AnalysisId']
    sourceanalysisArn=sourceanalysis['Analysis']['Arn']
    sourceanalysisname=sourceanalysis['Analysis']['Name']
    DataSetArns=sourceanalysis['Analysis']['DataSetArns']
    sourcetid=sourceanalysisid
    sourcetname=sourceanalysisname
    targettid=sourcetid
    targettname=sourceanalysisname
    
    TargetThemeArn=''
    if 'ThemeArn' in sourceanalysis['Analysis'].keys():
        SourceThemeArn=sourceanalysis['Analysis']['ThemeArn']
        TargetThemeArn = 'arn:aws:quicksight:'+region+':'+account_id+':theme/'+sourceanalysis['Analysis']['ThemeArn'].split("/")[1]

    sourcedsref = []
    for i in DataSetArns:
        missing=False
        did = i.split("/")[1]
        try:
            dname=get_dataset_name(did, sourcesession)
        except Exception as e:
            faillist.append({"Error Type": "Dataset: "+did+" is missing!","sourceanalysisid": sourcetid, "Name": sourcetname, "Error": str(e)})
            missing=True
            break
            
        sourcedsref.append({'DataSetPlaceholder': dname,
                    'DataSetArn': i})
    if missing: continue
    try:
        sourcetemplate = create_template(sourcesession, sourcetid, sourcetname, sourcedsref, sourceanalysisArn, '1')
        sourcetemplate=describe_template(sourcesession,sourcetid)
    except Exception as e:
        faillist.append({"Error Type": "Create Source Template Error","sourceanalysisid": sourcetid, "Name": sourcetname, "Error": str(e)})

        continue
        
    while sourcetemplate['Template']['Version']['Status']=="CREATION_IN_PROGRESS":
        time.sleep(5)
        sourcetemplate=describe_template(sourcesession,sourcetid)
        if sourcetemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
            try:
                updateres=update_template_permission(sourcesession, sourcetid, targetroot)
            except Exception as e:
                delete_template(sourcesession, sourcetid)
                faillist.append({"Error Type": "Update Source Template Permission Error",
                                 "sourceanalysisid": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
    else: 
        if sourcetemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
            try:
                updateres=update_template_permission(sourcesession, sourcetid, targetroot)
            except Exception as e:
                delete_template(sourcesession, sourcetid)
                faillist.append({"Error Type": "Update Source Template Permission Error",
                                 "sourceanalysisid": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
                continue          

    ds=data_sets (targetsession)
    Template=sourcetemplate['Template']
    dsref=[]
    
    missing=False
    for i in Template['Version']['DataSetConfigurations']:
    #print(i)
        n=Template['Version']['DataSetConfigurations'].index(i)
    #print(n)
        for j in ds:
            if i['Placeholder']==j['Name']:
                dsref.append({
                    'DataSetPlaceholder': i['Placeholder'],
                    'DataSetArn': j['Arn']
                })
            if n>len(dsref): 
                e="Dataset "+i['Placeholder']+"is missing!"
                faillist.append({"Error Type": "Datasets in target env are missing for this analysis",
                                 "sourceanalysisid": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
                missing=True
                break
        if missing: break
    if missing: continue
        
##working
    SourceEntity={
        'SourceTemplate': {
            'DataSetReferences': dsref,
            'Arn': Template['Arn']
        }
    }
    
    #print(SourceEntity)
    analysis=describe_analysis(targetsession, targettid)
    if 'Faild to describe analysis:' in analysis or analysis['Analysis']['Status']=='DELETED':
        if 'analysis/'+targettid+' is not found' in analysis or analysis['Analysis']['Status']=='DELETED':
            print("Create new anlaysis now:")
            try:
                newanalysis=create_analysis(targetsession, targettid, targettname,targetadmin,SourceEntity,TargetThemeArn)
            except Exception as e:
                delete_template(sourcesession, targettid)
                faillist.append({"Error Type": "Create New Analysis Error",
                                 "AnalysisID": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)}) 
                continue
        else:
            faillist.append({"Error Type": "Describe Target Analysis Error",
                                 "AnalysisID": targettid, 
                                 "Name": targettname, 
                                 "Error": str(analysis)}) 
            continue
    elif analysis['Analysis']['Status']=="CREATION_FAILED":
        res=delete_analysis(sourcesession, targettid)
        try:
            newanalysis=create_analysis(targetsession, targettid, targettname,targetadmin, SourceEntity,TargetThemeArn)
        except Exception as e:
            delete_template(sourcesession, targettid)
            faillist.append({"Error Type": "Create Analysis Error",
                                 "AnalysisID": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)})
            continue
            
    else:
        print("analysis is existing. update it now.")
        try:
            newanalysis=update_analysis(targetsession, targettid, targettname, SourceEntity,TargetThemeArn)
        except Exception as e:
            delete_template(sourcesession, targettid)
            faillist.append({"Error Type": "Update Analysis Error",
                                 "AnalysisID": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)})
            continue
    time.sleep(20)
    res=describe_analysis(targetsession,newanalysis['AnalysisId'])
    if res['Status']==200:
        status=res['Analysis']['Status']
        if status=='CREATION_SUCCESSFUL' or status=='UPDATE_SUCCESSFUL':
            success.append(res['Analysis'])
            #filename="Migration_Results/Successful/Analysis_"+res['Analysis']['Name']+".json"
        else:
            faillist.append({"Error Type": "Analysis Creation Status is not Successful", "Analysis": res['Analysis']})
            #filename="Migration_Results/Fail/Analysis_"+res['Analysis']['Name']+".json"

In [41]:
with open(faillocation+now+'Analysis_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

with open(successlocation+now+'Analysis_Success.json', "w") as f:
    json.dump(success, f, indent=4, sort_keys=True, default=str)

<strong>Get dashboards list</strong>

#source account
sourceaccountid=""
role_name=""
aws_region='us-east-1'
sourcesession = _assume_role(sourceaccountid, role_name, aws_region)

#target account
targetaccountid=""
role_name=""
aws_region='us-east-1'
targetsession = _assume_role(targetaccountid, role_name, aws_region)

In [42]:
sourcedashboards=dashboards(sourcesession)

#Get id of datasets migration list
migration_list=[]
for newset in dashboard_migrate_list:
        ids = get_dashboard_ids(newset, sourcesession)
        for dashboard in sourcedashboards:
            if ids[0] == dashboard["DashboardId"]:
                migration_list.append(dashboard)

JSON(migration_list)

<IPython.core.display.JSON object>

Migrate dashboards

In [43]:
success=[]
faillist=[]
for dashboard in migration_list:
    sourcedashboard=describe_dashboard(sourcesession, dashboard['DashboardId'])
    SourceEntityArn=sourcedashboard['Dashboard']['Version']['SourceEntityArn']
    print(SourceEntityArn)
    if SourceEntityArn.split("/")[0].split(":")[-1]=="analysis" or SourceEntityArn.split("/")[0].split(":")[-1]=='template':
        sourceanalysis=sourcedashboard['Dashboard']['Version']['SourceEntityArn']
    else: 
        faillist.append({"Error Type": "Source Analysis is missing!","DashboardId": sourcetid, "Name": sourcetname, "Error": "Source Analysis is missing!"})
        continue
    
    sourceversion=sourcedashboard['Dashboard']['Version']['VersionNumber']
    sourcedid=sourcedashboard['Dashboard']['DashboardId']
    sourcedname=sourcedashboard['Dashboard']['Name']
    sourcetid=sourcedid
    sourcetname=sourcedname
    targetdid=get_target_id(sourcetid)
    targetdname=get_target_id(sourcetname)
    print('1')
    
        
    DataSetArns=sourcedashboard['Dashboard']['Version']['DataSetArns']
    TargetThemeArn=''
    if 'ThemeArn' in sourcedashboard['Dashboard']['Version'].keys():
        SourceThemearn=sourcedashboard['Dashboard']['Version']['ThemeArn']
        TargetThemeArn = 'arn:aws:quicksight:'+region+':'+account_id+':theme/'+SourceThemearn.split("/")[1]
    sourcedsref = []
    print('2')
    for i in DataSetArns:
        missing=False
        did = i.split("/")[1]
        try:
            dname=get_dataset_name(did, sourcesession)
        except Exception as e:
            faillist.append({"Error Type": "Dataset: "+did+" is missing!","DashboardId": sourcetid, "Name": sourcetname, "Error": str(e)})
            missing=True
            break

        sourcedsref.append({'DataSetPlaceholder': dname,
                        'DataSetArn': i})
    if missing: continue
            
    if SourceEntityArn.split("/")[0].split(":")[-1]=="analysis" and same_account_migration == True:

        try:
            sourcetemplate = create_template(sourcesession, sourcetid, sourcetname, sourcedsref, sourceanalysis, '1')
            sourcetemplate = describe_template(sourcesession,sourcetid)
            print('template created')
        except Exception as e:
            faillist.append({"Error Type": "Create Source Template Error","DashboardId": sourcetid, "Name": sourcetname, "Error": str(e)})
            continue
        print('3')
        while sourcetemplate['Template']['Version']['Status']=="CREATION_IN_PROGRESS":
            time.sleep(5)
            sourcetemplate=describe_template(sourcesession,sourcetid)
            #print(sourcetemplate)

        print('4')
        sourcetemplate = describe_template(sourcesession,sourcetid)
        targettemplate = describe_template(targetsession,sourcetid) #there is no need to copy template within the same account

        while targettemplate['Template']['Version']['Status']=="CREATION_IN_PROGRESS":
            time.sleep(5)
            targettemplate=describe_template(targetsession,sourcetid)
            if targettemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
                break
        else: 
            if targettemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
                print("Template is successful copied!")
            else: 
                delete_template(targetsession, sourcetid)
                faillist.append({"Error Type": "Copy Template Error",
                                     "DashboardId": sourcetid, 
                                     "Name": sourcetname, 
                                     "Error": str(e)})
                continue

    elif SourceEntityArn.split("/")[0].split(":")[-1]=="template" and same_account_migration == True:
        sourcetid = SourceEntityArn.split("/")[1]
        sourcetemplate = describe_template(sourcesession,sourcetid)
        targettemplate = describe_template(targetsession,sourcetid)
        
    elif SourceEntityArn.split("/")[0].split(":")[-1]=="analysis" and same_account_migration == False:
        #To fill
        print('to do')
            
    elif SourceEntityArn.split("/")[0].split(":")[-1]=="analysis" and same_account_migration == False:
        #To fill
        print('to do')
        
        
    #ds=data_sets (targetsession)
    ds = folder_members (targetsession, target_folder_ID)
    Template=targettemplate['Template']
    dsref=[]
    #print(Template['Version']['DataSetConfigurations'])
    missing=False
    for i in Template['Version']['DataSetConfigurations']:
        #print("i is "+str(i))
        n=Template['Version']['DataSetConfigurations'].index(i)
        #print("n is "+str(n))
        for j in ds:
            member_type = j['MemberArn'].split('/')[0].split(':')[-1]
            if member_type != 'dataset':
                continue
            j['Arn'] = j['MemberArn']    
            ds_des = describe_dataset (targetsession, j['MemberId'])
            j['Name'] = ds_des['DataSet']['Name']
            if get_target_placeholder(i['Placeholder'])==j['Name']:
                print(i['Placeholder'])
                print(j['Name'])
                print(j['Arn'])
                dsref.append({
                    'DataSetPlaceholder': i['Placeholder'],
                    'DataSetArn': j['Arn']
                })
                break
                print("len of dsref is "+str(len(dsref)))
                print(dsref)
        if (n+1)>len(dsref): 
            e="Dataset "+i['Placeholder']+" is missing!"
            faillist.append({"Error Type": "Datasets in target env are missing for this dashboard",
                                 "DashboardId": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
            missing=True
            break
        if missing: break
    if missing: continue
    #print("len of dsref is "+str(len(dsref)))
    #print(dsref) 
    
    if target_is_dev == True: # create an analysis for dev purposes first
        print('create an analysis for dev purposes first')
        SourceEntity={
            'SourceTemplate': {
                'DataSetReferences': dsref,
                'Arn': Template['Arn']
            }
        }
        newanalysis=create_analysis(targetsession, targetdid, targetdname, targetadmin, SourceEntity,TargetThemeArn)
        #print(newanalysis)
        time.sleep(30)
        create_group_membership(targetsession, target_folder_ID, targetdid, 'ANALYSIS' )
        continue
        
    else:    
        SourceEntity={
            'SourceTemplate': {
                'DataSetReferences': dsref,
                'Arn': Template['Arn']
            }
        }
    #print(SourceEntity)
    dashboard=describe_dashboard(targetsession, targetdid)

    if 'Faild to describe dashboard:' in dashboard:
        if 'dashboard/'+targetdid+' is not found' in dashboard:
            print("Create new dashboard now:")
            try:
                newdashboard=create_dashboard(targetsession, targetdid, targetdname,targetadmin, SourceEntity, '1',TargetThemeArn, filter='DISABLED',csv='ENABLED', sheetcontrol='COLLAPSED')
            except Exception as e:
                delete_template(targetsession, targetdid)
                faillist.append({"Error Type": "Create New Dashboard Error",
                                 "DashboardId": targetdid, 
                                 "Name": targetdname, 
                                 "Error": str(e)}) 
                continue
        else: 
            faillist.append({"Error Type": "Describe Target Dashboard Error",
                                 "DashboardId": targetdid, 
                                 "Name": targetdname, 
                                 "Error": str(dashboard)}) 
            continue
    elif dashboard['Dashboard']['Version']['Status']=="CREATION_FAILED":
        res=delete_dashboard(targetsession, targetdid)
        try:
            newdashboard=create_dashboard(targetsession, targetdid, targetdname,targetadmin, SourceEntity, '1',TargetThemeArn, filter='DISABLED',csv='ENABLED', sheetcontrol='COLLAPSED')
        except Exception as e:
            delete_template(targetsession, targetdid)
            print('fail to create dashboard, add to faillist')
            faillist.append({"Error Type": "Create Dashboard Error",
                                 "DashboardId": targetdid, 
                                 "Name": targetdname, 
                                 "Error": str(e)})
            continue
            
    else:
        print("dashboard is existing. update it now.")
        try:
            res=delete_dashboard(targetsession, targetdid)
            newdashboard=create_dashboard(targetsession, targetdid, targetdname,targetadmin, SourceEntity, '1',TargetThemeArn, filter='DISABLED',csv='ENABLED', sheetcontrol='COLLAPSED')
        except Exception as e:
            #print(newdashboard)
            delete_template(targetsession, targetdid)
            print('fail to update dashboard, add to faillist')
            faillist.append({"Error Type": "Create Dashboard Error",
                                 "DashboardId": targetdid, 
                                 "Name": targetdname, 
                                 "Error": str(e)})
            continue
    while(True):
        res=describe_dashboard(targetsession,newdashboard['DashboardId'])

        if res['Status']==200:
            status=res['Dashboard']['Version']['Status']
            if status=='CREATION_SUCCESSFUL' or status=='UPDATE_SUCCESSFUL':
                success.append(res['Dashboard'])
                break
            elif status=='CREATION_IN_PROGRESS':
                time.sleep(10)
                continue
            else:
                faillist.append({"Error Type": "Dashboard Creation Status is not Successful", "Dashboard": res['Dashboard']})
                break
                #filename="Migration_Results/Fail/Dashboard_"+res['Dashboard']['Name']+".json"
            
    create_group_membership(targetsession, target_folder_ID, targetdid, 'DASHBOARD' )
    print('added dashboard to {} folder'.format(target_folder_name))

arn:aws:quicksight:us-east-1:233081732471:analysis/1c0f3212-d0f8-40bf-a344-e19ba9690b9b
1
2
template created
3
4
Template is successful copied!
qs_dataset_date_range
qs_dataset_date_range-UAT
arn:aws:quicksight:us-east-1:233081732471:dataset/a6a7c495-3174-4013-9766-d80a84eae54d-UAT
qs_fuel_comparison
qs_fuel_comparison-UAT
arn:aws:quicksight:us-east-1:233081732471:dataset/12bc8d1c-9158-4bd2-b5bc-8af977149b20-UAT
qs_location
qs_location-UAT
arn:aws:quicksight:us-east-1:233081732471:dataset/41dbfd62-bbc7-4516-ab6a-fa51278b9dca-UAT
qs_not_reporting
qs_not_reporting-UAT
arn:aws:quicksight:us-east-1:233081732471:dataset/f423b170-f4dc-436e-b58b-fdbf2d4654b1-UAT
qs_subscription
qs_subscription-UAT
arn:aws:quicksight:us-east-1:233081732471:dataset/0e91b4ef-4a53-495d-8c88-66560abed72e-UAT
qs_ecu
qs_ecu-UAT
arn:aws:quicksight:us-east-1:233081732471:dataset/f5506260-505b-418c-aa45-c9a04f606180-UAT
qs_location_comparison
qs_location_comparison-UAT
arn:aws:quicksight:us-east-1:233081732471:dataset/

In [44]:
with open(faillocation+now+'Dashboard_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

with open(successlocation+now+'Dashboard_Success.json', "w") as f:
    json.dump(success, f, indent=4, sort_keys=True, default=str)


In [45]:
faillist

[]

<strong>Delete objects<strong/>

# THIS WILL DELETE ALL TARGET DATASETS

delete = "template"

if delete == "datasource":   
    for datasource in data_sources(targetsession):
        #if datasource['Type'] == "REDSHIFT":
        try:
            delete_source (targetsession, datasource['DataSourceId'])
        except Exception: pass 
elif delete == "dataset":
    for dataset in data_sets(targetsession):
        delete_dataset (targetsession, dataset['DataSetId'])
elif delete == "template":    
    for template in templates(targetsession):
        delete_template(targetsession, template['TemplateId'])
elif delete == "analysis":
    for analysis in analysis(targetsession): delete_analysis(targetsession, analysis['AnalysisId'])
    
elif delete == "dashboard":    
    for dashboard in dashboards(targetsession):
        delete_dashboard(targetsession, dashboard['DashboardId'])
delete ="don't delete anything"

Schedule notebooks to execute

https://aws.amazon.com/blogs/machine-learning/scheduling-jupyter-notebooks-on-sagemaker-ephemeral-instances/